### Project Title:  Convolutional Neural Networks for Computer Vision (Python)
Key Notes: 
1.	Implementing Backpropagation algorithm.
2.	Model works as image classifier.
3.	convolution, max pooling and average pooling using NumPy array operations


Given the function of the form:

$f(x,y)=\frac{x+\sigma(y)}{\sigma(x)+(x+y)^2 }$

The sigmoid function $\sigma(x)$ is defined as:

$\sigma(x)=\frac{1}{1+e^{-x}}=\frac{e^x}{e^x+1}$

Its gradient is:

$\frac{\partial\sigma(x)}{\partial x}=(1-\sigma(x))\sigma(x)$



### Define the computational graph

Hint1: We will define the computational graph for each node. The CG for the given function is then a combination of all nodes.

Hint2: Each node should be defined as a class. Then we can have multiple instances of it.

#### Define a class for addition

In [1]:
# The first one is an example for you
# No need to change it
class Addition():
    '''A simple addition node'''
    
    # The __init__ method is alwayed required for a class definition.
    # It defines how to create an instance of this class
    # It will be called each time an instance of this class is created
    def __init__(self):
        self.result = 0
        self.gradient = 0
        
    def forward(self, x, y):
        # the keyword self refers to the address of the instance
        # a variable self.xx belongs to the instance
        # it can be called be all methods of the class once initialized
        # therefore you can use them for the backward operation
        self.x = x
        self.y = y
        self.result = self.x + self.y
        return self.result
    
    def backward(self, upstream_gradient):
        return upstream_gradient # because local gradient = 1

#### Define a class for reciprocal

In [2]:
# Finish the class Reciprocal
class Reciprocal():
    '''A simple reciprocal node'''
    
    def __init__(self):
        self.result = 1 # not possible to have reciprocal=0
        self.gradient = 0
        
    def forward(self, x):
        self.x = x
        self.result = 1 / x
        return self.result
    
    def backward(self, upstream_gradient):
        local_gradient = -1 / (self.x * self.x)
        self.gradient = upstream_gradient * local_gradient
        return self.gradient

#### Define a class for square

In [3]:
class Square():
    '''A simple square gate'''
    
    def __init__(self):
        self.result = 0
        self.gradient = 0
        
    def forward(self, x):
        self.x = x
        self.result = self.x * self.x
        return self.result
    
    def backward(self, upstream_gradient):
        local_gradient = 2 * self.x
        self.gradient = upstream_gradient * local_gradient
        return self.gradient

#### Define a class for sigmoid

In [4]:
import math # we need to use math.exp()

class Sigmoid():
    '''A simple sigmoid gate'''
    
    def __init__(self):
        self.result = 0.5
        self.gradient = 0
        
    def forward(self, x):
        self.x = x
        self.result = 1 / (1 + math.exp(-x))
        return self.result
    
    def backward(self, upstream_gradient):
        local_gradient = (1 - self.result) * self.result
        self.gradient = upstream_gradient * local_gradient
        return self.gradient

#### Define a class for multiplication

In [5]:
class Multiplication():
    '''A simple mul gate'''
    
    def __init__(self):
        self.result = 0
        self.gradient_x = 0
        self.gradient_y = 0
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        self.result = self.x * self.y
        return self.result
    
    def backward(self, upstream_gradient):
        local_gradient_x = self.y
        local_gradient_y = self.x
        # gradients are different for the inputs
        self.gradient = [upstream_gradient * local_gradient_x, upstream_gradient * local_gradient_y]
        return self.gradient

#### Define the class for the complete computational graph

In [6]:
class ComputationalGraph():
    '''The computational graph and its forward and backward functions'''
    
    def __init__(self):
        # create the whole graph by creating instances of the above defined classes
        # since we want to use them for the forward and backward operation, they must have self keyword
        # for example...
        self.sigm1 = Sigmoid()
        # fill out the rest, refer to the computational graph you have plotted for 1.a
        self.add1 = Addition()
        self.sq1 = Square()
        self.add2 = Addition()
        self.rec1 = Reciprocal()
        self.sigm2 = Sigmoid()
        self.add3 = Addition()
        self.mul1 = Multiplication()
    
    def forward(self, x, y):
        # now connect all the nodes
        self.x = x
        self.y = y
        
        z0 = self.sigm1.forward(self.x)
        
        z1 = self.add1.forward(self.x, self.y)
        z1 = self.sq1.forward(z1)
        z1 = self.add2.forward(z0, z1)
        z1 = self.rec1.forward(z1)
        
        z2 = self.sigm2.forward(self.y)
        z2 = self.add3.forward(self.x, z2)
        
        self.result = self.mul1.forward(z1, z2)
        
        return self.result
        
        
    def backward(self):
        upstream_gradient = 1
        
        grad_mul1 = self.mul1.backward(upstream_gradient)
        
        grad_rec1 = self.rec1.backward(grad_mul1[0])
        grad_add2 = self.add2.backward(grad_rec1)
        grad_sigm1 = self.sigm1.backward(grad_add2)
        
        grad_sq1 = self.sq1.backward(grad_add2)
        grad_add1 = self.add1.backward(grad_sq1)
        
        grad_add3 = self.add3.backward(grad_mul1[1])
        grad_sigm2 = self.sigm2.backward(grad_add3)
        
        self.grad_x = grad_sigm1 + grad_add1 + grad_add3
        self.grad_y = grad_add1 + grad_sigm2
        
        return [self.grad_x, self.grad_y]

### Use the defined graph to do some actual computation

In [7]:
x = 3
y = -4
my_graph = ComputationalGraph()

result = my_graph.forward(x, y)
print('The result of the function is {}'.format(result))

gradients = my_graph.backward()
print('The gradients for x and y are {}'.format(gradients))

The result of the function is 1.5456448841066441
The gradients for x and y are [2.0595697955721652, 1.5922327514838093]
